In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores

##Best config is 180: {'topK': 494, 'alpha': 0.1597863887003789}

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13936 ( 0.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13921 ( 0.5%) Users that have less than 1 test interactions


In [4]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [5]:
### USERCF Recommender
ItemCF_recommender = UserKNNCFRecommender(URM_train=URM_train)
UserCF_recommender.fit(topK=450, shrink=5.0, similarity="tanimoto")

UserKNNCFRecommender: URM Detected 3 ( 0.0%) users with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 13984 (100.0%), 1069.62 column/sec. Elapsed time 13.07 sec


In [6]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

100%|████████████████████████████████████▉| 18032/18059 [12:22<00:00, 58.59it/s]

In [7]:
similarity_userCF = UserCF_recommender.W_sparse
similarity_slime = SLIME_recommender.W_sparse

In [8]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real,Integer

hyperparameters_range_dictionary = {
    "topK": Integer(10,500),
    "alpha": Real(0.0, 1.0)
}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, similarity_userCF, similarity_slime],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=ItemKNNSimilarityHybridRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=ItemKNNSimilarityHybridRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )




Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 489, 'alpha': 0.4734188095309392}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 0 Exception. Config: {'topK': 489, 'alpha': 0.4734188095309392} - Exception: Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 301, in _fit_model
    

Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 301, in _fit_model
    recommender_instance = self.recommender_class(*self.recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS,
  File "/Users/gabriele/PycharmProjects/RecSys/src/Recommenders/KNN/ItemKNNSimilarityHybridRecommender.py", line 27, in __init__
    raise ValueError("ItemKNNSimilarityHybridRecommender: similarities have different size, S1 is {},

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2321
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 73, 'alpha': 0.654313734523202}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 1 Exception. Config: {'topK': 73, 'alpha': 0.654313734523202} - Exception: Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperpa

Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 301, in _fit_model
    recommender_instance = self.recommender_class(*self.recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS,
  File "/Users/gabriele/PycharmProjects/RecSys/src/Recommenders/KNN/ItemKNNSimilarityHybridRecommender.py", line 27, in __init__
    raise ValueError("ItemKNNSimilarityHybridRecommender: similarities have different size, S1 is {},

Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0735
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 5 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 117, 'alpha': 0.2920965927562286}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 4 Exception. Config: {'topK': 117, 'alpha': 0.2920965927562286} - Exception: Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyp

...

Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 301, in _fit_model
    recommender_instance = self.recommender_class(*self.recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS,
  File "/Users/gabriele/PycharmProjects/RecSys/src/Recommenders/KNN/ItemKNNSimilarityHybridRecommender.py", line 27, in __init__
    raise ValueError("ItemKNNSimilarityHybridRecommender: similarities have different size, S1 is {},

Iteration No: 55 ended. Evaluation done at random point.
Time taken: 0.0710
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 56 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 131, 'alpha': 0.8729925219430907}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 55 Exception. Config: {'topK': 131, 'alpha': 0.8729925219430907} - Exception: Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_

Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 301, in _fit_model
    recommender_instance = self.recommender_class(*self.recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS,
  File "/Users/gabriele/PycharmProjects/RecSys/src/Recommenders/KNN/ItemKNNSimilarityHybridRecommender.py", line 27, in __init__
    raise ValueError("ItemKNNSimilarityHybridRecommender: similarities have different size, S1 is {},

Iteration No: 58 ended. Evaluation done at random point.
Time taken: 0.0691
Function value obtained: 65504.0000
Current minimum: 65504.0000
Iteration No: 59 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 446, 'alpha': 0.3452089220792261}
ItemKNNSimilarityHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.
SearchBayesianSkopt: Config 58 Exception. Config: {'topK': 446, 'alpha': 0.3452089220792261} - Exception: Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_

Traceback (most recent call last):
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 464, in _objective_function
    result_df, recommender_instance = self._evaluate_on_validation(current_fit_hyperparameters_dict, was_already_evaluated_flag, was_already_evaluated_index)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 332, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_hyperparameters)
  File "/Users/gabriele/PycharmProjects/RecSys/src/HyperparameterTuning/SearchAbstractClass.py", line 301, in _fit_model
    recommender_instance = self.recommender_class(*self.recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS,
  File "/Users/gabriele/PycharmProjects/RecSys/src/Recommenders/KNN/ItemKNNSimilarityHybridRecommender.py", line 27, in __init__
    raise ValueError("ItemKNNSimilarityHybridRecommender: similarities have different size, S1 is {},